## Population projections statistics

Notebook environment to migrate TIF files to CF compliant CoG

In [65]:
# Import standard packages
import os
import glob
import pathlib
import sys
import numpy as np
import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import xarray as xr
from dotenv import load_dotenv
import math
from pathlib import Path
from typing import Any, Dict, List, Optional, Tuple, Union
import rioxarray as rio
#load_dotenv()

# Import custom functionality
from coclicodata.drive_config import p_drive
from coclicodata.etl.cf_compliancy_checker import check_compliancy, save_compliancy
from coastmonitor.io.utils import name_block

# Define (local and) remote drives
coclico_data_dir = p_drive.joinpath("11207608-coclico", "FULLTRACK_DATA")

# Workaround to the Windows OS (10) udunits error after installation of cfchecker: https://github.com/SciTools/iris/issues/404
os.environ["UDUNITS2_XML_PATH"] = str(
    pathlib.Path().home().joinpath(  # change to the udunits2.xml file dir in your Python installation
        r"Anaconda3\pkgs\udunits2-2.2.28-h892ecd3_0\Library\share\udunits\udunits2.xml"
    )
)

# use local or remote data dir
use_local_data = False
ds_dirname = "WP5"

if use_local_data: 
    ds_dir = pathlib.Path().home().joinpath("data", ds_dirname)
else: 
    ds_dir = coclico_data_dir.joinpath(ds_dirname, "data", "pop_fp_statistics")

if not ds_dir.exists():
    raise FileNotFoundError("Directory with data does not exist.")


In [73]:
parquet_file = ds_dir.joinpath("pop_fp_LAU.parquet")

pop_stats = gpd.read_parquet(parquet_file)

pop_stats

,GISCO_ID,CNTR_CODE,nuts_2,nuts_3,LAU_ID,LAU_NAME,POP_2020,POP_DENS_2,AREA_KM2,YEAR,...,UNDEFENDED_MAPS\1\High_End\2150\population_SSP5,UNDEFENDED_MAPS\SLR_High_end\2150\population_SSP1,UNDEFENDED_MAPS\SLR_High_end\2150\population_SSP2,UNDEFENDED_MAPS\SLR_High_end\2150\population_SSP5,UNDEFENDED_MAPS\static\High_End\2150\population_SSP1,UNDEFENDED_MAPS\static\High_End\2150\population_SSP2,UNDEFENDED_MAPS\static\High_End\2150\population_SSP5,tot_pop\2150\population_SSP1,tot_pop\2150\population_SSP2,tot_pop\2150\population_SSP5
0,DE_03361001,DE,DE93,DE93B,03361001,"Achim, Stadt",31923.0,471.398771,67.719735,2020,...,NaN,0.144375,0.179375,NaN,NaN,NaN,0.752165,14295.223125,17457.186875,63640.814859
1,DE_03361002,DE,DE93,DE93B,03361002,Blender,2885.0,75.034640,38.448908,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DE_03361003,DE,DE93,DE93B,03361003,Dörverden,9009.0,107.940908,83.462333,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DE_03361004,DE,DE93,DE93B,03361004,Emtinghausen,1464.0,68.573846,21.349247,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DE_03361005,DE,DE93,DE93B,03361005,Kirchlinteln,9911.0,56.675823,174.871744,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11960,FR_97614,FR,FRY5,FRY50,97614,Ouangani,10203.0,555.838786,18.356042,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11961,FR_97616,FR,FRY5,FRY50,97616,Sada,11156.0,1012.643086,11.016715,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11962,FR_97617,FR,FRY5,FRY50,97617,Tsingoni,13934.0,406.428213,34.284037,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11963,FR_97608,FR,FRY5,FRY50,97608,Dzaoudzi,17831.0,2797.974566,6.372824,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [74]:
# Print all column names in for loop
sort_col = []

for col in pop_stats.columns:
    # print(col)
    sort_col.append(col)

sort_col.sort()
sort_col


['AREA_KM2',
 'CNTR_CODE',
 'FID',
 'GISCO_ID',
 'HIGH_DEFENDED_MAPS\\1000\\High_End\\2150\\population_SSP1',
 'HIGH_DEFENDED_MAPS\\1000\\High_End\\2150\\population_SSP2',
 'HIGH_DEFENDED_MAPS\\1000\\High_End\\2150\\population_SSP5',
 'HIGH_DEFENDED_MAPS\\1000\\High_End\\B01\\2100\\population_SSP1',
 'HIGH_DEFENDED_MAPS\\1000\\High_End\\B01\\2100\\population_SSP2',
 'HIGH_DEFENDED_MAPS\\1000\\High_End\\B01\\2100\\population_SSP5',
 'HIGH_DEFENDED_MAPS\\1000\\None\\2010\\population_SSP1',
 'HIGH_DEFENDED_MAPS\\1000\\None\\2010\\population_SSP2',
 'HIGH_DEFENDED_MAPS\\1000\\None\\2010\\population_SSP5',
 'HIGH_DEFENDED_MAPS\\1000\\SSP126\\B01\\2100\\population_SSP1',
 'HIGH_DEFENDED_MAPS\\1000\\SSP126\\B01\\2100\\population_SSP2',
 'HIGH_DEFENDED_MAPS\\1000\\SSP126\\B01\\2100\\population_SSP5',
 'HIGH_DEFENDED_MAPS\\1000\\SSP245\\2050\\population_SSP1',
 'HIGH_DEFENDED_MAPS\\1000\\SSP245\\2050\\population_SSP2',
 'HIGH_DEFENDED_MAPS\\1000\\SSP245\\2050\\population_SSP5',
 'HIGH_DEFENDED_

In [75]:
# Get rid of the 'B01' element
pop_stats.columns = pop_stats.columns.str.replace('\\B01', '', regex=False)

# Print all column names in for loop
for col in pop_stats.columns:
    print(col)

GISCO_ID
CNTR_CODE
nuts_2
nuts_3
LAU_ID
LAU_NAME
POP_2020
POP_DENS_2
AREA_KM2
YEAR
FID
geometry
HIGH_DEFENDED_MAPS\1000\None\2010\population_SSP1
HIGH_DEFENDED_MAPS\1000\None\2010\population_SSP2
HIGH_DEFENDED_MAPS\1000\None\2010\population_SSP5
HIGH_DEFENDED_MAPS\100\None\2010\population_SSP1
HIGH_DEFENDED_MAPS\100\None\2010\population_SSP2
HIGH_DEFENDED_MAPS\100\None\2010\population_SSP5
HIGH_DEFENDED_MAPS\1\None\2010\population_SSP1
HIGH_DEFENDED_MAPS\1\None\2010\population_SSP2
HIGH_DEFENDED_MAPS\1\None\2010\population_SSP5
HIGH_DEFENDED_MAPS\static\None\2010\population_SSP1
HIGH_DEFENDED_MAPS\static\None\2010\population_SSP2
HIGH_DEFENDED_MAPS\static\None\2010\population_SSP5
LOW_DEFENDED_MAPS\1000\None\2010\population_SSP1
LOW_DEFENDED_MAPS\1000\None\2010\population_SSP2
LOW_DEFENDED_MAPS\1000\None\2010\population_SSP5
LOW_DEFENDED_MAPS\100\None\2010\population_SSP1
LOW_DEFENDED_MAPS\100\None\2010\population_SSP2
LOW_DEFENDED_MAPS\100\None\2010\population_SSP5
LOW_DEFENDED_MAPS\1

In [76]:
pop_stats_red = pop_stats

# There are columns that represent combinations of population SSP and Flood map SSP that can be dropped
for col in pop_stats_red.columns:
    
    print(col)
    # split col at backslash
    vars = col.split("\\")

    # Find outlying naming of columns
    if 'MAPS' in vars[0] and len(vars) < 5:
                
        # Drop column
        pop_stats_red.drop(columns=[col], inplace=True)

        continue

    # Match floodmap ssp with population ssp
    if 'MAPS' in vars[0] and not vars[-1].split('_')[-1] in vars[2] and not (vars[2] == 'None' and "SSP2" in vars[-1]) and not (vars[2] == 'High_End' and "SSP5" in vars[-1]):

        # Drop column
        pop_stats_red.drop(columns=[col], inplace=True)

# Print all column names in for loop
for col in pop_stats_red.columns:
    print(col)

GISCO_ID
CNTR_CODE
nuts_2
nuts_3
LAU_ID
LAU_NAME
POP_2020
POP_DENS_2
AREA_KM2
YEAR
FID
geometry
HIGH_DEFENDED_MAPS\1000\None\2010\population_SSP1
HIGH_DEFENDED_MAPS\1000\None\2010\population_SSP2
HIGH_DEFENDED_MAPS\1000\None\2010\population_SSP5
HIGH_DEFENDED_MAPS\100\None\2010\population_SSP1
HIGH_DEFENDED_MAPS\100\None\2010\population_SSP2
HIGH_DEFENDED_MAPS\100\None\2010\population_SSP5
HIGH_DEFENDED_MAPS\1\None\2010\population_SSP1
HIGH_DEFENDED_MAPS\1\None\2010\population_SSP2
HIGH_DEFENDED_MAPS\1\None\2010\population_SSP5
HIGH_DEFENDED_MAPS\static\None\2010\population_SSP1
HIGH_DEFENDED_MAPS\static\None\2010\population_SSP2
HIGH_DEFENDED_MAPS\static\None\2010\population_SSP5
LOW_DEFENDED_MAPS\1000\None\2010\population_SSP1
LOW_DEFENDED_MAPS\1000\None\2010\population_SSP2
LOW_DEFENDED_MAPS\1000\None\2010\population_SSP5
LOW_DEFENDED_MAPS\100\None\2010\population_SSP1
LOW_DEFENDED_MAPS\100\None\2010\population_SSP2
LOW_DEFENDED_MAPS\100\None\2010\population_SSP5
LOW_DEFENDED_MAPS\1

In [77]:
# Move all columns containing "tot_pop" to column position 11
# Identify columns starting with 'tot_pop'
tot_pop_cols = [col for col in pop_stats_red.columns if col.startswith('tot_pop')]

# Identify remaining columns
other_cols = [col for col in pop_stats_red.columns if col not in tot_pop_cols]

# New column order: insert tot_pop columns at the specified location (e.g., index 11)
# Adjust the insertion index (e.g., 11) based on the size of your DataFrame
insert_index = min(12, len(other_cols))  # Ensures the index is valid
new_column_order = other_cols[:insert_index] + tot_pop_cols + other_cols[insert_index:]

# Reorder columns
pop_stats_red = pop_stats_red[new_column_order]

for col in pop_stats_red.columns:
    print(col)  

GISCO_ID
CNTR_CODE
nuts_2
nuts_3
LAU_ID
LAU_NAME
POP_2020
POP_DENS_2
AREA_KM2
YEAR
FID
geometry
tot_pop\2010\population_SSP1
tot_pop\2010\population_SSP2
tot_pop\2010\population_SSP5
tot_pop\2030\population_SSP1
tot_pop\2030\population_SSP2
tot_pop\2030\population_SSP5
tot_pop\2050\population_SSP1
tot_pop\2050\population_SSP2
tot_pop\2050\population_SSP5
tot_pop\2100\population_SSP1
tot_pop\2100\population_SSP2
tot_pop\2100\population_SSP5
tot_pop\2150\population_SSP1
tot_pop\2150\population_SSP2
tot_pop\2150\population_SSP5
HIGH_DEFENDED_MAPS\1000\None\2010\population_SSP2
HIGH_DEFENDED_MAPS\100\None\2010\population_SSP2
HIGH_DEFENDED_MAPS\1\None\2010\population_SSP2
HIGH_DEFENDED_MAPS\static\None\2010\population_SSP2
LOW_DEFENDED_MAPS\1000\None\2010\population_SSP2
LOW_DEFENDED_MAPS\100\None\2010\population_SSP2
LOW_DEFENDED_MAPS\1\None\2010\population_SSP2
LOW_DEFENDED_MAPS\static\None\2010\population_SSP2
UNDEFENDED_MAPS\1000\None\2010\population_SSP2
UNDEFENDED_MAPS\100\None\2010\

In [78]:
# Add columns for the population statistics
# Suffixes to append
suffixes = ["\\abs_affected", "\\rel_affected", "\\abs_unaffected", "\\rel_unaffected"]

# Identify columns containing "MAPS"
maps_columns = [col for col in pop_stats_red.columns if "MAPS" in col]

# Generate expanded columns for "MAPS"
expanded_columns = [f"{col}{suffix}" for col in maps_columns for suffix in suffixes]

# Retain original columns (non-MAPS)
non_maps_columns = [col for col in pop_stats_red.columns if "MAPS" not in col]

# Create new DataFrame structure
new_columns = non_maps_columns + expanded_columns
pop_stats_rel = pd.DataFrame(columns=new_columns)

# Copy original non-MAPS data to the new DataFrame
pop_stats_rel[non_maps_columns] = pop_stats_red[non_maps_columns]

for col in pop_stats_rel.columns:
    print(col)  

GISCO_ID
CNTR_CODE
nuts_2
nuts_3
LAU_ID
LAU_NAME
POP_2020
POP_DENS_2
AREA_KM2
YEAR
FID
geometry
tot_pop\2010\population_SSP1
tot_pop\2010\population_SSP2
tot_pop\2010\population_SSP5
tot_pop\2030\population_SSP1
tot_pop\2030\population_SSP2
tot_pop\2030\population_SSP5
tot_pop\2050\population_SSP1
tot_pop\2050\population_SSP2
tot_pop\2050\population_SSP5
tot_pop\2100\population_SSP1
tot_pop\2100\population_SSP2
tot_pop\2100\population_SSP5
tot_pop\2150\population_SSP1
tot_pop\2150\population_SSP2
tot_pop\2150\population_SSP5
HIGH_DEFENDED_MAPS\1000\None\2010\population_SSP2\abs_affected
HIGH_DEFENDED_MAPS\1000\None\2010\population_SSP2\rel_affected
HIGH_DEFENDED_MAPS\1000\None\2010\population_SSP2\abs_unaffected
HIGH_DEFENDED_MAPS\1000\None\2010\population_SSP2\rel_unaffected
HIGH_DEFENDED_MAPS\100\None\2010\population_SSP2\abs_affected
HIGH_DEFENDED_MAPS\100\None\2010\population_SSP2\rel_affected
HIGH_DEFENDED_MAPS\100\None\2010\population_SSP2\abs_unaffected
HIGH_DEFENDED_MAPS\100\No

In [79]:
# Iterate over all columns of pop_stats_rel containing "MAPS"	
for col in maps_columns:
    # Iterate over all columns containing "tot_pop"
    for tot_pop_col in tot_pop_cols:

        # Get column name variables
        vars = col.split("\\")
        tot_pop_vars = tot_pop_col.split("\\")
    
        if vars[-1].split('_')[-1] in tot_pop_vars[-1] and vars[3] in tot_pop_vars[1]:

            # Extract the total population
            tot_pop_cur = pop_stats_red[tot_pop_col].values

            # Extract population affected
            pop_aff_cur = pop_stats_red[col].values

            # Compute the absolute number of affected people
            abs_affected = pop_aff_cur

            # Compute the relative number of affected people
            rel_affected = pop_aff_cur / tot_pop_cur

            # Compute the absolute number of unaffected people
            abs_unaffected = tot_pop_cur - pop_aff_cur

            # Compute the relative number of unaffected people
            rel_unaffected = abs_unaffected / tot_pop_cur

            # Append the computed values to the new DataFrame
            pop_stats_rel[col + suffixes[0]] = abs_affected
            pop_stats_rel[col + suffixes[1]] = rel_affected
            pop_stats_rel[col + suffixes[2]] = abs_unaffected
            pop_stats_rel[col + suffixes[3]] = rel_unaffected

# Convert the DataFrame to a GeoDataFrame
pop_stats_rel = gpd.GeoDataFrame(pop_stats_rel, geometry=pop_stats_red.geometry)

C:\Users\soest\AppData\Local\Temp\ipykernel_14248\693051340.py:22: RuntimeWarning: invalid value encountered in divide
  rel_affected = pop_aff_cur / tot_pop_cur
C:\Users\soest\AppData\Local\Temp\ipykernel_14248\693051340.py:28: RuntimeWarning: invalid value encountered in divide
  rel_unaffected = abs_unaffected / tot_pop_cur


In [80]:
# Check if pop_stat_rel is still a geodataframe
if isinstance(pop_stats_rel, gpd.GeoDataFrame):
    print("pop_stats_rel is a geodataframe")

pop_stats_rel is a geodataframe


In [81]:
# Write parquet file
# parquet_file = ds_dir.joinpath("pop_fp_LAU_rel.parquet")
# pop_stats_rel.to_parquet(parquet_file)

In [82]:
# Rename the FID column to 'LAU_FID'
# pop_stats_rel.rename(columns={'FID': 'LAU_FID'}, inplace=True)

# # Also write to gpkg for front-end purposes
# gpkg_file = ds_dir.joinpath("pop_fp_LAU.gpkg")
# pop_stats_rel.to_file(str(gpkg_file), layer='pp_LAU', driver='GPKG')

In [83]:
for col in pop_stats_rel.select_dtypes(include=['object']).columns:
    print(col, pop_stats_rel[col].apply(type).value_counts())

GISCO_ID GISCO_ID
<class 'str'>    11965
Name: count, dtype: int64
CNTR_CODE CNTR_CODE
<class 'str'>    11965
Name: count, dtype: int64
nuts_2 nuts_2
<class 'str'>    11965
Name: count, dtype: int64
nuts_3 nuts_3
<class 'str'>    11965
Name: count, dtype: int64
LAU_ID LAU_ID
<class 'str'>    11965
Name: count, dtype: int64
LAU_NAME LAU_NAME
<class 'str'>    11965
Name: count, dtype: int64
FID FID
<class 'str'>    11965
Name: count, dtype: int64


In [84]:
# Write to geopackage
gpkg_file = ds_dir.joinpath("pop_fp_LAU.gpkg")
pop_stats_test.to_file(str(gpkg_file), layer='pp_LAU_rel', driver='GPKG')

NameError: name 'pop_stats_test' is not defined

In [85]:
# Print all column names
for col in pop_stats_rel.columns:
    print(col)

GISCO_ID
CNTR_CODE
nuts_2
nuts_3
LAU_ID
LAU_NAME
POP_2020
POP_DENS_2
AREA_KM2
YEAR
FID
geometry
tot_pop\2010\population_SSP1
tot_pop\2010\population_SSP2
tot_pop\2010\population_SSP5
tot_pop\2030\population_SSP1
tot_pop\2030\population_SSP2
tot_pop\2030\population_SSP5
tot_pop\2050\population_SSP1
tot_pop\2050\population_SSP2
tot_pop\2050\population_SSP5
tot_pop\2100\population_SSP1
tot_pop\2100\population_SSP2
tot_pop\2100\population_SSP5
tot_pop\2150\population_SSP1
tot_pop\2150\population_SSP2
tot_pop\2150\population_SSP5
HIGH_DEFENDED_MAPS\1000\None\2010\population_SSP2\abs_affected
HIGH_DEFENDED_MAPS\1000\None\2010\population_SSP2\rel_affected
HIGH_DEFENDED_MAPS\1000\None\2010\population_SSP2\abs_unaffected
HIGH_DEFENDED_MAPS\1000\None\2010\population_SSP2\rel_unaffected
HIGH_DEFENDED_MAPS\100\None\2010\population_SSP2\abs_affected
HIGH_DEFENDED_MAPS\100\None\2010\population_SSP2\rel_affected
HIGH_DEFENDED_MAPS\100\None\2010\population_SSP2\abs_unaffected
HIGH_DEFENDED_MAPS\100\No

In [96]:
# Print all column names
for col in pop_stats_rel.columns:
    print(col)

# Check number of columns that do not contain "MAPS"
non_maps_cols = [col for col in pop_stats_rel.columns if "MAPS" not in col]
len(non_maps_cols)

GISCO_ID
CNTR_CODE
nuts_2
nuts_3
LAU_ID
LAU_NAME
POP_2020
POP_DENS_2
AREA_KM2
YEAR
FID
geometry
tot_pop\2010\population_SSP1
tot_pop\2010\population_SSP2
tot_pop\2010\population_SSP5
tot_pop\2030\population_SSP1
tot_pop\2030\population_SSP2
tot_pop\2030\population_SSP5
tot_pop\2050\population_SSP1
tot_pop\2050\population_SSP2
tot_pop\2050\population_SSP5
tot_pop\2100\population_SSP1
tot_pop\2100\population_SSP2
tot_pop\2100\population_SSP5
tot_pop\2150\population_SSP1
tot_pop\2150\population_SSP2
tot_pop\2150\population_SSP5
HIGH_DEFENDED_MAPS\1000\None\2010\population_SSP2\abs_affected
HIGH_DEFENDED_MAPS\1000\None\2010\population_SSP2\rel_affected
HIGH_DEFENDED_MAPS\1000\None\2010\population_SSP2\abs_unaffected
HIGH_DEFENDED_MAPS\1000\None\2010\population_SSP2\rel_unaffected
HIGH_DEFENDED_MAPS\100\None\2010\population_SSP2\abs_affected
HIGH_DEFENDED_MAPS\100\None\2010\population_SSP2\rel_affected
HIGH_DEFENDED_MAPS\100\None\2010\population_SSP2\abs_unaffected
HIGH_DEFENDED_MAPS\100\No

27

In [87]:
#path_prefix = "file:///opt/coclico-data-public/coclico/"
path_prefix = Path(r"file:/opt/coclico-data-public/coclico/")
#file://C:/data/cp_cfhp/HIGH_DEFENDED_MAPS/Mean_spring_tide
 
 
local_path_with_files = Path(r"P:\11207608-coclico\FULLTRACK_DATA\WP4\cogs_28nov24\UNDEFENDED_MAPS\1000\High_End\2100")
 
#read in the local_path_with_files directory all the paths that contains files and keep the unique paths
# paths = []
# for root, dirs, files in os.walk(local_path_with_files):
#     for file in files:
#         paths.append(os.path.join(root, file))
 
# #remove the files from the paths
# paths = [os.path.dirname(path) for path in paths]
# #append at the beginning of the path the path_prefix
 
# paths = [path.replace('P:\\11207608-coclico\\FULLTRACK_DATA\\WP4\\cogs_28nov24\\UNDEFENDED_MAPS\\1000\\High_End\\2100', 'file:///opt/coclico-data-public/coclico/cfhp_all/') for path in paths]
# paths = [path_prefix + path for path in paths]
# # make all the '\' to '/'
# paths = [path.replace('\\', '/') for path in paths]
# unique_paths = list(set(paths))
 
# Join path of prefix with last 4 parts of local path
path_base = path_prefix.joinpath(*local_path_with_files.parts[-4:])

# combine all .tif files from local directory with path_base
tif_files = list(local_path_with_files.glob('*.tif'))

paths = []
for tif_file in tif_files:
    # append the path_base with the tif_file name
    paths.append(path_base.joinpath(tif_file.name))

# Convert paths to a list of strings keeping forward slashes
paths = [str(path.as_posix()) for path in paths]
paths

['file:/opt/coclico-data-public/coclico/UNDEFENDED_MAPS/1000/High_End/2100/B01_epsg=3035_x=1547762_y=1319937.tif',
 'file:/opt/coclico-data-public/coclico/UNDEFENDED_MAPS/1000/High_End/2100/B01_epsg=3035_x=1547762_y=2139137.tif',
 'file:/opt/coclico-data-public/coclico/UNDEFENDED_MAPS/1000/High_End/2100/B01_epsg=3035_x=1547762_y=2958337.tif',
 'file:/opt/coclico-data-public/coclico/UNDEFENDED_MAPS/1000/High_End/2100/B01_epsg=3035_x=1547762_y=3777537.tif',
 'file:/opt/coclico-data-public/coclico/UNDEFENDED_MAPS/1000/High_End/2100/B01_epsg=3035_x=1547762_y=4596737.tif',
 'file:/opt/coclico-data-public/coclico/UNDEFENDED_MAPS/1000/High_End/2100/B01_epsg=3035_x=1547762_y=941637.tif',
 'file:/opt/coclico-data-public/coclico/UNDEFENDED_MAPS/1000/High_End/2100/B01_epsg=3035_x=2366962_y=1319937.tif',
 'file:/opt/coclico-data-public/coclico/UNDEFENDED_MAPS/1000/High_End/2100/B01_epsg=3035_x=2366962_y=2139137.tif',
 'file:/opt/coclico-data-public/coclico/UNDEFENDED_MAPS/1000/High_End/2100/B01_ep

In [88]:
import shapely

# Open parquet file from google bucket URL
# URL of the Parquet file
url = "https://storage.googleapis.com/coclico-data-public/coclico/pp_stats/pop_fp_LAU_rel.parquet"

# Read the Parquet file
pp_stats = pd.read_parquet(url, engine='pyarrow')

# Because we load with regular pandas the polygon data needs to be converted from WKB - Well-Known Binary to shapely.Polygon
pp_stats['geometry'] = pp_stats['geometry'].apply(shapely.wkb.loads)

# Now convert to geopandas
pp_stats = gpd.GeoDataFrame(pp_stats, geometry='geometry', crs='EPSG:3035')

pp_stats

,GISCO_ID,CNTR_CODE,nuts_2,nuts_3,LAU_ID,LAU_NAME,POP_2020,POP_DENS_2,AREA_KM2,YEAR,...,UNDEFENDED_MAPS\100\High_End\2150\population_SSP5\abs_unaffected,UNDEFENDED_MAPS\100\High_End\2150\population_SSP5\rel_unaffected,UNDEFENDED_MAPS\1\High_End\2150\population_SSP5\abs_affected,UNDEFENDED_MAPS\1\High_End\2150\population_SSP5\rel_affected,UNDEFENDED_MAPS\1\High_End\2150\population_SSP5\abs_unaffected,UNDEFENDED_MAPS\1\High_End\2150\population_SSP5\rel_unaffected,UNDEFENDED_MAPS\static\High_End\2150\population_SSP5\abs_affected,UNDEFENDED_MAPS\static\High_End\2150\population_SSP5\rel_affected,UNDEFENDED_MAPS\static\High_End\2150\population_SSP5\abs_unaffected,UNDEFENDED_MAPS\static\High_End\2150\population_SSP5\rel_unaffected
0,DE_03361001,DE,DE93,DE93B,03361001,"Achim, Stadt",31923.0,471.398771,67.719735,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,0.752165,0.000012,63640.062694,0.999988
1,DE_03361002,DE,DE93,DE93B,03361002,Blender,2885.0,75.034640,38.448908,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,DE_03361003,DE,DE93,DE93B,03361003,Dörverden,9009.0,107.940908,83.462333,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DE_03361004,DE,DE93,DE93B,03361004,Emtinghausen,1464.0,68.573846,21.349247,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,DE_03361005,DE,DE93,DE93B,03361005,Kirchlinteln,9911.0,56.675823,174.871744,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11960,FR_97614,FR,FRY5,FRY50,97614,Ouangani,10203.0,555.838786,18.356042,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11961,FR_97616,FR,FRY5,FRY50,97616,Sada,11156.0,1012.643086,11.016715,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11962,FR_97617,FR,FRY5,FRY50,97617,Tsingoni,13934.0,406.428213,34.284037,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
11963,FR_97608,FR,FRY5,FRY50,97608,Dzaoudzi,17831.0,2797.974566,6.372824,2020,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [125]:
# List all columns containing "MAPS"
maps_columns = [str(Path(col).parent) for col in pop_stats_red.columns if "MAPS" in col]
len(maps_columns)

108

In [124]:
for map_column in maps_columns:
    
    # Select all column names containing the current map column
    cur_stats = pp_stats.loc[:, pp_stats.columns.str.startswith(map_column)]

    # Add the cur stats to the first 11 columns of pp_stats
    pp_gpkg = pd.concat([pp_stats.iloc[:, :12], cur_stats], axis=1)

    pp_gpkg.rename(columns={'FID': 'LAU_FID'}, inplace=True)

    # Set geopackage file name
    scen_name = map_column.replace("\\", "_")
    gpkg_file = ds_dir.joinpath("geopackages", f"pp_stats_{scen_name}.gpkg")

    # Write to geopackage
    pp_gpkg.to_file(str(gpkg_file), layer=f'pp_stats_{scen_name}', driver='GPKG')

In [123]:
pp_gpkg

,GISCO_ID,CNTR_CODE,nuts_2,nuts_3,LAU_ID,LAU_NAME,POP_2020,POP_DENS_2,AREA_KM2,YEAR,FID,geometry,HIGH_DEFENDED_MAPS\1000\None\2010\population_SSP2\abs_affected,HIGH_DEFENDED_MAPS\1000\None\2010\population_SSP2\rel_affected,HIGH_DEFENDED_MAPS\1000\None\2010\population_SSP2\abs_unaffected,HIGH_DEFENDED_MAPS\1000\None\2010\population_SSP2\rel_unaffected
0,DE_03361001,DE,DE93,DE93B,03361001,"Achim, Stadt",31923.0,471.398771,67.719735,2020,DE_03361001,"POLYGON ((4258927.295 3320011.519, 4259193.583...",2.574719,0.000060,42936.306129,0.999940
1,DE_03361002,DE,DE93,DE93B,03361002,Blender,2885.0,75.034640,38.448908,2020,DE_03361002,"POLYGON ((4264556.862 3317435.819, 4265677.354...",0.132360,0.000082,1618.922537,0.999918
2,DE_03361003,DE,DE93,DE93B,03361003,Dörverden,9009.0,107.940908,83.462333,2020,DE_03361003,"POLYGON ((4275479.269 3295636.790, 4275328.930...",0.000000,0.000000,5213.400932,1.000000
3,DE_03361004,DE,DE93,DE93B,03361004,Emtinghausen,1464.0,68.573846,21.349247,2020,DE_03361004,"POLYGON ((4250468.027 3309420.666, 4250432.180...",NaN,NaN,NaN,NaN
4,DE_03361005,DE,DE93,DE93B,03361005,Kirchlinteln,9911.0,56.675823,174.871744,2020,DE_03361005,"POLYGON ((4272425.673 3310568.711, 4274136.202...",NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
11960,FR_97614,FR,FRY5,FRY50,97614,Ouangani,10203.0,555.838786,18.356042,2020,FR_97614,"POLYGON ((8727685.023 -2785490.190, 8727139.84...",NaN,NaN,NaN,NaN
11961,FR_97616,FR,FRY5,FRY50,97616,Sada,11156.0,1012.643086,11.016715,2020,FR_97616,"POLYGON ((8727114.505 -2787670.745, 8725517.75...",NaN,NaN,NaN,NaN
11962,FR_97617,FR,FRY5,FRY50,97617,Tsingoni,13934.0,406.428213,34.284037,2020,FR_97617,"POLYGON ((8730964.436 -2773602.954, 8730972.75...",NaN,NaN,NaN,NaN
11963,FR_97608,FR,FRY5,FRY50,97608,Dzaoudzi,17831.0,2797.974566,6.372824,2020,FR_97608,"MULTIPOLYGON (((8745852.102 -2769772.861, 8742...",NaN,NaN,NaN,NaN
